In [20]:
# Imports e instalações

import pandas as pd
import numpy as np

# Viz
import matplotlib.pyplot as plt
import seaborn as sns

#!pip install scikit-learn
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer


# Modelos
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# Métricas
from sklearn.metrics import (accuracy_score, precision_score, recall_score, roc_auc_score, confusion_matrix, classification_report, f1_score)
from sklearn.metrics import RocCurveDisplay, PrecisionRecallDisplay

In [21]:
# Carregando o DF

# df recebe o dataset
df = pd.read_csv(r'C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_editados\australia_clima_v5.csv', sep=',')
df

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna,choveu_hoje,amplitude_termica,umidade_media,variacao_pressao
0,2007-11-01,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,False,False,False,False,False,False,0,16.3,48.5,4.7
1,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,False,False,False,False,False,False,1,12.9,58.0,4.0
2,2007-11-03,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,False,False,False,False,False,False,1,9.7,75.5,2.3
3,2007-11-04,Canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,...,False,False,False,False,False,False,1,2.2,59.0,-1.5
4,2007-11-05,Canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,...,False,False,False,False,False,False,1,8.5,58.5,-0.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56565,2017-06-25,Perth,6.3,17.0,0.0,1.6,7.9,E,26.0,SE,...,False,False,False,False,False,False,0,10.7,62.0,2.6
56566,2017-06-25,Watsonia,7.9,13.0,0.0,2.8,3.8,NNW,39.0,N,...,False,False,False,False,False,False,0,5.1,68.5,2.3
56567,2017-06-25,Sydney,7.6,19.3,0.0,3.4,9.4,W,35.0,W,...,False,False,False,False,False,False,0,11.7,52.5,3.2
56568,2017-06-25,PerthAirport,4.9,16.8,0.0,1.6,7.9,E,39.0,E,...,False,False,False,False,False,False,0,11.9,56.5,2.5


In [22]:
# Verificação de tamanho do DF
df.shape

(56570, 37)

In [23]:
# Verificação tipos de dados
df.dtypes

data                        object
localidade                  object
temp_min                   float64
temp_max                   float64
chuva_mm                   float64
evaporacao                 float64
horas_sol                  float64
vento_rajada_direcao        object
vento_rajada_velocidade    float64
vento_direcao_9h            object
vento_direcao_15h           object
vento_veloc_9h             float64
vento_veloc_15h            float64
umidade_9h                 float64
umidade_15h                float64
pressao_9h                 float64
pressao_15h                float64
nuvem_9h                   float64
nuvem_15h                  float64
temp_9h                    float64
temp_15h                   float64
choveu_hoje_fex             object
chove_amanha_vtr             int64
ano                          int64
mes                          int64
dia                          int64
trimestre_quarter            int64
horas_sol_isna                bool
evaporacao_isna     

In [24]:
# Verificação de valores nulos
df.isnull().sum()

data                       0
localidade                 0
temp_min                   0
temp_max                   0
chuva_mm                   0
evaporacao                 0
horas_sol                  0
vento_rajada_direcao       0
vento_rajada_velocidade    0
vento_direcao_9h           0
vento_direcao_15h          0
vento_veloc_9h             0
vento_veloc_15h            0
umidade_9h                 0
umidade_15h                0
pressao_9h                 0
pressao_15h                0
nuvem_9h                   0
nuvem_15h                  0
temp_9h                    0
temp_15h                   0
choveu_hoje_fex            0
chove_amanha_vtr           0
ano                        0
mes                        0
dia                        0
trimestre_quarter          0
horas_sol_isna             0
evaporacao_isna            0
nuvem_9h_isna              0
nuvem_15h_isna             0
pressao_9h_isna            0
pressao_15h_isna           0
choveu_hoje                0
amplitude_term

In [25]:
# Convertendo a coluna de data para datetime e ordenando do mais antigo para o mais recente // split temporal
df['data'] = pd.to_datetime(df['data'], errors='coerce')
df = df.sort_values(by='data').reset_index(drop=True)
df.dtypes, display(df.head(3))

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna,choveu_hoje,amplitude_termica,umidade_media,variacao_pressao
0,2007-11-01,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,False,False,False,False,False,False,0,16.3,48.5,4.7
1,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,False,False,False,False,False,False,1,12.9,58.0,4.0
2,2007-11-03,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,False,False,False,False,False,False,1,9.7,75.5,2.3


(data                       datetime64[ns]
 localidade                         object
 temp_min                          float64
 temp_max                          float64
 chuva_mm                          float64
 evaporacao                        float64
 horas_sol                         float64
 vento_rajada_direcao               object
 vento_rajada_velocidade           float64
 vento_direcao_9h                   object
 vento_direcao_15h                  object
 vento_veloc_9h                    float64
 vento_veloc_15h                   float64
 umidade_9h                        float64
 umidade_15h                       float64
 pressao_9h                        float64
 pressao_15h                       float64
 nuvem_9h                          float64
 nuvem_15h                         float64
 temp_9h                           float64
 temp_15h                          float64
 choveu_hoje_fex                    object
 chove_amanha_vtr                    int64
 ano       

In [26]:
df.describe(include='all')

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,horas_sol_isna,evaporacao_isna,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna,choveu_hoje,amplitude_termica,umidade_media,variacao_pressao
count,56570,56570,56570.000000,56570.000000,56570.000000,56570.000000,56570.000000,56570,56570.000000,56570,...,56570,56570,56570,56570,56570,56570,56570.000000,56570.000000,56570.000000,56570.000000
unique,NaN,26,NaN,NaN,NaN,NaN,NaN,16,NaN,16,...,1,1,1,1,1,1,NaN,NaN,NaN,NaN
top,NaN,Darwin,NaN,NaN,NaN,NaN,NaN,E,NaN,N,...,False,False,False,False,False,False,NaN,NaN,NaN,NaN
freq,NaN,3063,NaN,NaN,NaN,NaN,NaN,4531,NaN,4981,...,56570,56570,56570,56570,56570,56570,NaN,NaN,NaN,NaN
mean,2012-09-16 14:43:51.302810880,NaN,13.458626,24.210032,2.131868,5.499295,7.732842,NaN,40.866343,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.220983,10.751405,57.753597,2.443039
min,2007-11-01 00:00:00,NaN,-6.700000,4.100000,0.000000,0.000000,0.000000,NaN,9.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,-3.200000,0.000000,-10.800000
25%,2010-07-18 00:00:00,NaN,8.600000,18.700000,0.000000,2.800000,5.000000,NaN,31.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,7.100000,47.000000,1.400000
50%,2012-07-27 00:00:00,NaN,13.200000,23.900000,0.000000,5.000000,8.600000,NaN,39.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,10.100000,59.000000,2.700000
75%,2014-10-09 00:00:00,NaN,18.400000,29.700000,0.600000,7.400000,10.700000,NaN,48.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,14.000000,70.000000,3.700000
max,2017-06-25 00:00:00,NaN,31.400000,48.100000,206.200000,81.200000,14.500000,NaN,124.000000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000,31.400000,100.000000,15.100000


In [27]:
# Criação da var mes_sin
# transformando o mes em um valor cíclico usando a função seno // entre -1 e 1 // isso faz com que o modelo entenda a ciclicidade dos meses do ano // dezembro e janeiro estão próximos
df['mes_sin'] = np.sin(2 * np.pi * df['data'].dt.month / 12)

# Criação da var mes_cos
# transformando o mes em um valor cíclico usando a função cosseno // entre -1 e 1 // isso faz com que o modelo entenda a ciclicidade dos meses do ano // dezembro e janeiro estão próximos
df['mes_cos'] = np.cos(2 * np.pi * df['mes'] / 12)
df.head(3)
#df.isna().sum()

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna,choveu_hoje,amplitude_termica,umidade_media,variacao_pressao,mes_sin,mes_cos
0,2007-11-01,Canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,False,False,False,False,0,16.3,48.5,4.7,-0.5,0.866025
1,2007-11-02,Canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,False,False,False,False,1,12.9,58.0,4.0,-0.5,0.866025
2,2007-11-03,Canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,False,False,False,False,1,9.7,75.5,2.3,-0.5,0.866025


In [28]:
# Trazendo longitude e latitude das cidades australianas
df_worldcities = pd.read_csv(r'C:\Users\JacyzinGuilherme(Bip\mentoria-bip\dados_brutos\worldcities.csv', sep=',')

In [29]:
# Verificação do DF worldcities
df_worldcities

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6870,139.7495,Japan,JP,JPN,Tōkyō,primary,37785000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629
...,...,...,...,...,...,...,...,...,...,...,...
48054,Al Jabīn,Al Jabin,14.7040,43.5990,Yemen,YE,YEM,Raymah,admin,NaN,1887910100
48055,Nelspruit,Nelspruit,-25.4745,30.9703,South Africa,ZA,ZAF,Mpumalanga,admin,NaN,1710114438
48056,Gqeberha,Gqeberha,-33.9681,25.5981,South Africa,ZA,ZAF,Eastern Cape,NaN,NaN,1710000082
48057,Lupane,Lupane,-18.9315,27.8070,Zimbabwe,ZW,ZWE,Matabeleland North,admin,NaN,1716206606


In [30]:
# Refinando DF apenas com cidades da Austrália
df_cidades_au = df_worldcities[df_worldcities['country'] == 'Australia']


In [31]:
# Conferindo o df_cidades_au
df_cidades_au

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
127,Melbourne,Melbourne,-37.8142,144.9631,Australia,AU,AUS,Victoria,admin,5031195.0,1036533631
134,Sydney,Sydney,-33.8667,151.2000,Australia,AU,AUS,New South Wales,admin,4840600.0,1036074917
295,Brisbane,Brisbane,-27.4678,153.0281,Australia,AU,AUS,Queensland,admin,2706966.0,1036192929
388,Perth,Perth,-31.9558,115.8597,Australia,AU,AUS,Western Australia,admin,2141834.0,1036178956
545,Adelaide,Adelaide,-34.9275,138.6000,Australia,AU,AUS,South Australia,admin,1387290.0,1036538171
...,...,...,...,...,...,...,...,...,...,...,...
47725,Bedourie,Bedourie,-24.3594,139.4691,Australia,AU,AUS,Queensland,NaN,122.0,1036680924
47727,Mount Isa,Mount Isa,-20.7261,139.4939,Australia,AU,AUS,Queensland,NaN,118.0,1036386100
47731,Birdsville,Birdsville,-25.8986,139.3513,Australia,AU,AUS,Queensland,NaN,110.0,1036830335
47734,Windorah,Windorah,-25.4205,142.6544,Australia,AU,AUS,Queensland,NaN,104.0,1036272226


In [32]:
# Reservando as variáveis importantes do dataset
df_cidades_au = df_cidades_au[['city', 'lat', 'lng']]
df_cidades_au.rename(columns={'city': 'cidade', 'lat': 'latitude', 'lng': 'longitude'}, inplace=True)
df_cidades_au.reset_index(drop=True, inplace=True)

C:\Users\JacyzinGuilherme(Bip\AppData\Local\Temp\ipykernel_14612\3334688593.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cidades_au.rename(columns={'city': 'cidade', 'lat': 'latitude', 'lng': 'longitude'}, inplace=True)


In [33]:
# Colocando todas as cidades em lowercase para facilitar o merge
df['localidade'] = df['localidade'].str.lower()
df['localidade'] = df['localidade'].str.strip()
df['localidade'] = df['localidade'].str.replace(r'\s+', ' ', regex=True).str.strip()


df_cidades_au['cidade'] = df_cidades_au['cidade'].str.lower()
df_cidades_au['cidade'] = df_cidades_au['cidade'].str.strip()
df_cidades_au['cidade'] = df_cidades_au['cidade'].str.replace(r'\s+', ' ', regex=True).str.strip()

C:\Users\JacyzinGuilherme(Bip\AppData\Local\Temp\ipykernel_14612\3981580434.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cidades_au['cidade'] = df_cidades_au['cidade'].str.lower()
C:\Users\JacyzinGuilherme(Bip\AppData\Local\Temp\ipykernel_14612\3981580434.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cidades_au['cidade'] = df_cidades_au['cidade'].str.strip()
C:\Users\JacyzinGuilherme(Bip\AppData\Local\Temp\ipykernel_14612\3981580434.py:9: SettingWithCopyWarning: 
A value is trying to be s

In [34]:
df

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,nuvem_9h_isna,nuvem_15h_isna,pressao_9h_isna,pressao_15h_isna,choveu_hoje,amplitude_termica,umidade_media,variacao_pressao,mes_sin,mes_cos
0,2007-11-01,canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,False,False,False,False,0,16.3,48.5,4.7,-5.000000e-01,0.866025
1,2007-11-02,canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,False,False,False,False,1,12.9,58.0,4.0,-5.000000e-01,0.866025
2,2007-11-03,canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,False,False,False,False,1,9.7,75.5,2.3,-5.000000e-01,0.866025
3,2007-11-04,canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,...,False,False,False,False,1,2.2,59.0,-1.5,-5.000000e-01,0.866025
4,2007-11-05,canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,...,False,False,False,False,1,8.5,58.5,-0.2,-5.000000e-01,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56565,2017-06-25,sydneyairport,6.8,19.5,0.0,3.4,9.4,WSW,46.0,NW,...,False,False,False,False,0,12.7,54.5,3.2,1.224647e-16,-1.000000
56566,2017-06-25,perth,6.3,17.0,0.0,1.6,7.9,E,26.0,SE,...,False,False,False,False,0,10.7,62.0,2.6,1.224647e-16,-1.000000
56567,2017-06-25,watsonia,7.9,13.0,0.0,2.8,3.8,NNW,39.0,N,...,False,False,False,False,0,5.1,68.5,2.3,1.224647e-16,-1.000000
56568,2017-06-25,sydney,7.6,19.3,0.0,3.4,9.4,W,35.0,W,...,False,False,False,False,0,11.7,52.5,3.2,1.224647e-16,-1.000000


In [35]:
df_cidades_au

,cidade,latitude,longitude
0,melbourne,-37.8142,144.9631
1,sydney,-33.8667,151.2000
2,brisbane,-27.4678,153.0281
3,perth,-31.9558,115.8597
4,adelaide,-34.9275,138.6000
...,...,...,...
269,bedourie,-24.3594,139.4691
270,mount isa,-20.7261,139.4939
271,birdsville,-25.8986,139.3513
272,windorah,-25.4205,142.6544


In [36]:
# Realizando o merge entre os dois dataframes com base na coluna de cidade/localidade // how='Left' para manter todas as linhas do df original
df = df.merge(df_cidades_au, left_on='localidade', right_on='cidade', how='left')
df

,data,localidade,temp_min,temp_max,chuva_mm,evaporacao,horas_sol,vento_rajada_direcao,vento_rajada_velocidade,vento_direcao_9h,...,pressao_15h_isna,choveu_hoje,amplitude_termica,umidade_media,variacao_pressao,mes_sin,mes_cos,cidade,latitude,longitude
0,2007-11-01,canberra,8.0,24.3,0.0,3.4,6.3,NW,30.0,SW,...,False,0,16.3,48.5,4.7,-5.000000e-01,0.866025,canberra,-35.2931,149.1269
1,2007-11-02,canberra,14.0,26.9,3.6,4.4,9.7,ENE,39.0,E,...,False,1,12.9,58.0,4.0,-5.000000e-01,0.866025,canberra,-35.2931,149.1269
2,2007-11-03,canberra,13.7,23.4,3.6,5.8,3.3,NW,85.0,N,...,False,1,9.7,75.5,2.3,-5.000000e-01,0.866025,canberra,-35.2931,149.1269
3,2007-11-04,canberra,13.3,15.5,39.8,7.2,9.1,NW,54.0,WNW,...,False,1,2.2,59.0,-1.5,-5.000000e-01,0.866025,canberra,-35.2931,149.1269
4,2007-11-05,canberra,7.6,16.1,2.8,5.6,10.6,SSE,50.0,SSE,...,False,1,8.5,58.5,-0.2,-5.000000e-01,0.866025,canberra,-35.2931,149.1269
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56565,2017-06-25,sydneyairport,6.8,19.5,0.0,3.4,9.4,WSW,46.0,NW,...,False,0,12.7,54.5,3.2,1.224647e-16,-1.000000,NaN,NaN,NaN
56566,2017-06-25,perth,6.3,17.0,0.0,1.6,7.9,E,26.0,SE,...,False,0,10.7,62.0,2.6,1.224647e-16,-1.000000,perth,-31.9558,115.8597
56567,2017-06-25,watsonia,7.9,13.0,0.0,2.8,3.8,NNW,39.0,N,...,False,0,5.1,68.5,2.3,1.224647e-16,-1.000000,NaN,NaN,NaN
56568,2017-06-25,sydney,7.6,19.3,0.0,3.4,9.4,W,35.0,W,...,False,0,11.7,52.5,3.2,1.224647e-16,-1.000000,sydney,-33.8667,151.2000


In [37]:
# Verificação dos valores nulos após o merge
df.isnull().sum()

data                           0
localidade                     0
temp_min                       0
temp_max                       0
chuva_mm                       0
evaporacao                     0
horas_sol                      0
vento_rajada_direcao           0
vento_rajada_velocidade        0
vento_direcao_9h               0
vento_direcao_15h              0
vento_veloc_9h                 0
vento_veloc_15h                0
umidade_9h                     0
umidade_15h                    0
pressao_9h                     0
pressao_15h                    0
nuvem_9h                       0
nuvem_15h                      0
temp_9h                        0
temp_15h                       0
choveu_hoje_fex                0
chove_amanha_vtr               0
ano                            0
mes                            0
dia                            0
trimestre_quarter              0
horas_sol_isna                 0
evaporacao_isna                0
nuvem_9h_isna                  0
nuvem_15h_

In [38]:
# Tratando os valores nulos

In [ ]:
df